In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
original_df = pd.read_parquet('../data/silver/water_consumption_silver.parquet')
df = original_df.copy()

In [3]:
def create_lag_features(input_df):
    df = input_df.copy()

    all_features = []
    non_weather_features = []

    lag_features = {}

    for lag in [1, 2, 3, 6, 12, 24, 26, 48, 72]:
        
        lag_features[f'{lag}_hours_ago_input_flow_rate_first'] = df['input_flow_rate_first'].shift(lag)
        lag_features[f'{lag}_hours_agot_input_flow_rate_last'] = df['input_flow_rate_last'].shift(lag)
        lag_features[f'{lag}_hours_ago_input_flow_rate_mean'] = df['input_flow_rate_mean'].shift(lag)
        lag_features[f'{lag}_hours_ago_reservoir_level_percentage_first'] = df['reservoir_level_percentage_first'].shift(lag)
        lag_features[f'{lag}_hours_ago_reservoir_level_percentage_last'] = df['reservoir_level_percentage_last'].shift(lag)
        lag_features[f'{lag}_hours_ago_reservoir_level_percentage_mean'] = df['reservoir_level_percentage_mean'].shift(lag)
        lag_features[f'{lag}_hours_ago_output_flow_rate_first'] = df['output_flow_rate_first'].shift(lag)
        lag_features[f'{lag}_hours_ago_output_flow_rate_last'] = df['output_flow_rate_last'].shift(lag)
        lag_features[f'{lag}_hours_ago_output_flow_rate_mean'] = df['output_flow_rate_mean'].shift(lag)
        lag_features[f'{lag}_hours_ago_pressure_first'] = df['pressure_first'].shift(lag)
        lag_features[f'{lag}_hours_ago_pressure_last'] = df['pressure_last'].shift(lag)
        lag_features[f'{lag}_hours_ago_pressure_mean'] = df['pressure_mean'].shift(lag)
        lag_features[f'{lag}_hours_agot_pump_1_duration_sum'] = df['pump_1_duration_sum'].shift(lag)
        lag_features[f'{lag}_hours_agot_pump_2_duration_sum'] = df['pump_2_duration_sum'].shift(lag)
        lag_features[f'{lag}_hours_ago_temperature'] = df['air_temp_c'].shift(lag)
        lag_features[f'{lag}_hours_ago_precipitation'] = df['total_precip_mm'].shift(lag)
        lag_features[f'{lag}_hours_ago_humidity'] = df['relative_humidity_percentage'].shift(lag)

        all_features.extend([
            f'{lag}_hours_ago_input_flow_rate_first', f'{lag}_hours_agot_input_flow_rate_last', f'{lag}_hours_ago_input_flow_rate_mean',
            f'{lag}_hours_ago_reservoir_level_percentage_first', f'{lag}_hours_ago_reservoir_level_percentage_last', f'{lag}_hours_ago_reservoir_level_percentage_mean',
            f'{lag}_hours_ago_output_flow_rate_first', f'{lag}_hours_ago_output_flow_rate_last', f'{lag}_hours_ago_output_flow_rate_mean',
            f'{lag}_hours_ago_pressure_first', f'{lag}_hours_ago_pressure_last', f'{lag}_hours_ago_pressure_mean',
            f'{lag}_hours_agot_pump_1_duration_sum', f'{lag}_hours_agot_pump_2_duration_sum',
            f'{lag}_hours_ago_temperature', f'{lag}_hours_ago_precipitation', f'{lag}_hours_ago_humidity'
        ])

        non_weather_features.extend([
            f'{lag}_hours_ago_input_flow_rate_first', f'{lag}_hours_agot_input_flow_rate_last', f'{lag}_hours_ago_input_flow_rate_mean',
            f'{lag}_hours_ago_reservoir_level_percentage_first', f'{lag}_hours_ago_reservoir_level_percentage_last', f'{lag}_hours_ago_reservoir_level_percentage_mean',
            f'{lag}_hours_ago_output_flow_rate_first', f'{lag}_hours_ago_output_flow_rate_last', f'{lag}_hours_ago_output_flow_rate_mean',
            f'{lag}_hours_ago_pressure_first', f'{lag}_hours_ago_pressure_last', f'{lag}_hours_ago_pressure_mean',
            f'{lag}_hours_agot_pump_1_duration_sum', f'{lag}_hours_agot_pump_2_duration_sum'
        ])

    # Use pd.concat to concatenate the new lag features into the dataframe
    df = pd.concat([df, pd.DataFrame(lag_features)], axis=1)

    return df, all_features, non_weather_features

In [4]:
def create_window_features(input_df):
    df = input_df.copy()
    
    all_features = []
    non_weather_features = []

    window_features = {}

    for window in [2, 3, 6, 12, 24, 36, 48, 72]:
        window_features[f'{window}_hours_rolling_input_flow_rate_diff'] = df['input_flow_rate_diff'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0] if len(x) == window else pd.NA)
        window_features[f'{window}_hours_rolling_output_flow_rate_diff'] = df['output_flow_rate_diff'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0] if len(x) == window else pd.NA)
        window_features[f'{window}_hours_rolling_reservoir_level_change'] = df['reservoir_level_change'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0] if len(x) == window else pd.NA)
        window_features[f'{window}_hours_rolling_pressure_change'] = df['pressure_change'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0] if len(x) == window else pd.NA)
        window_features[f'{window}_hours_rolling_change_from_last_hour_output_flow_rate_mean'] = df['change_from_last_hour_output_flow_rate_mean'].rolling(window).apply(lambda x: x.iloc[-1] - x.iloc[0] if len(x) == window else pd.NA)
        window_features[f'{window}_hours_rolling_temperature'] = df['air_temp_c'].rolling(window).mean()
        window_features[f'{window}_hours_rolling_precipitation'] = df['total_precip_mm'].rolling(window).mean()
        window_features[f'{window}_hours_rolling_humidity'] = df['relative_humidity_percentage'].rolling(window).mean()
        
        all_features.extend([
            f'{window}_hours_rolling_input_flow_rate_diff', f'{window}_hours_rolling_output_flow_rate_diff', f'{window}_hours_rolling_reservoir_level_change',
            f'{window}_hours_rolling_pressure_change', f'{window}_hours_rolling_change_from_last_hour_output_flow_rate_mean', f'{window}_hours_rolling_temperature',
            f'{window}_hours_rolling_precipitation', f'{window}_hours_rolling_humidity'
        ])

        non_weather_features.extend([
            f'{window}_hours_rolling_input_flow_rate_diff', f'{window}_hours_rolling_output_flow_rate_diff', f'{window}_hours_rolling_reservoir_level_change',
            f'{window}_hours_rolling_pressure_change', f'{window}_hours_rolling_change_from_last_hour_output_flow_rate_mean'
        ])
    
    # Use pd.concat to concatenate the new window features into the dataframe
    df = pd.concat([df, pd.DataFrame(window_features)], axis=1)
    
    return df, all_features, non_weather_features

In [5]:
def create_targets(input_df):
    df = input_df.copy()
    
    targets = []
    for i in range(1, 25):  
        df[f'target_{i}'] = df['change_from_last_hour_output_flow_rate_mean'].shift(-i)
        targets.append(f'target_{i}')
        
    return df, targets

In [6]:
def create_training_dataset(input_df):
    df = input_df.copy()
    
    df, all_lag_features, non_weather_lag_features = create_lag_features(df)
    df, all_window_features, non_weather_window_features = create_window_features(df)
    df, targets = create_targets(df)
    df.dropna(inplace=True)
    
    date_features = ['timestamp', 'hour', 'day_of_week', 'week_of_year', 'year']
    all_features = date_features + all_lag_features + all_window_features
    all_non_weather_features = date_features + non_weather_lag_features + non_weather_window_features
    all_training_columns = all_features + targets
    
    return df[all_training_columns].reset_index(drop=True), all_features, all_non_weather_features

training_df, all_features, all_non_weather_features = create_training_dataset(df)

In [7]:
training_df.head(1)

,timestamp,hour,day_of_week,week_of_year,year,1_hours_ago_input_flow_rate_first,1_hours_agot_input_flow_rate_last,1_hours_ago_input_flow_rate_mean,1_hours_ago_reservoir_level_percentage_first,1_hours_ago_reservoir_level_percentage_last,1_hours_ago_reservoir_level_percentage_mean,1_hours_ago_output_flow_rate_first,1_hours_ago_output_flow_rate_last,1_hours_ago_output_flow_rate_mean,1_hours_ago_pressure_first,1_hours_ago_pressure_last,1_hours_ago_pressure_mean,1_hours_agot_pump_1_duration_sum,1_hours_agot_pump_2_duration_sum,1_hours_ago_temperature,1_hours_ago_precipitation,1_hours_ago_humidity,2_hours_ago_input_flow_rate_first,2_hours_agot_input_flow_rate_last,2_hours_ago_input_flow_rate_mean,2_hours_ago_reservoir_level_percentage_first,2_hours_ago_reservoir_level_percentage_last,2_hours_ago_reservoir_level_percentage_mean,2_hours_ago_output_flow_rate_first,2_hours_ago_output_flow_rate_last,2_hours_ago_output_flow_rate_mean,2_hours_ago_pressure_first,2_hours_ago_pressure_last,2_hours_ago_pressure_mean,2_hours_agot_pump_1_duration_sum,2_hours_agot_pump_2_duration_sum,2_hours_ago_temperature,2_hours_ago_precipitation,2_hours_ago_humidity,3_hours_ago_input_flow_rate_first,3_hours_agot_input_flow_rate_last,3_hours_ago_input_flow_rate_mean,3_hours_ago_reservoir_level_percentage_first,3_hours_ago_reservoir_level_percentage_last,3_hours_ago_reservoir_level_percentage_mean,3_hours_ago_output_flow_rate_first,3_hours_ago_output_flow_rate_last,3_hours_ago_output_flow_rate_mean,3_hours_ago_pressure_first,3_hours_ago_pressure_last,3_hours_ago_pressure_mean,3_hours_agot_pump_1_duration_sum,3_hours_agot_pump_2_duration_sum,3_hours_ago_temperature,3_hours_ago_precipitation,3_hours_ago_humidity,6_hours_ago_input_flow_rate_first,6_hours_agot_input_flow_rate_last,6_hours_ago_input_flow_rate_mean,6_hours_ago_reservoir_level_percentage_first,6_hours_ago_reservoir_level_percentage_last,6_hours_ago_reservoir_level_percentage_mean,6_hours_ago_output_flow_rate_first,6_hours_ago_output_flow_rate_last,6_hours_ago_output_flow_rate_mean,6_hours_ago_pressure_first,6_hours_ago_pressure_last,6_hours_ago_pressure_mean,6_hours_agot_pump_1_duration_sum,6_hours_agot_pump_2_duration_sum,6_hours_ago_temperature,6_hours_ago_precipitation,6_hours_ago_humidity,12_hours_ago_input_flow_rate_first,12_hours_agot_input_flow_rate_last,12_hours_ago_input_flow_rate_mean,12_hours_ago_reservoir_level_percentage_first,12_hours_ago_reservoir_level_percentage_last,12_hours_ago_reservoir_level_percentage_mean,12_hours_ago_output_flow_rate_first,12_hours_ago_output_flow_rate_last,12_hours_ago_output_flow_rate_mean,12_hours_ago_pressure_first,12_hours_ago_pressure_last,12_hours_ago_pressure_mean,12_hours_agot_pump_1_duration_sum,12_hours_agot_pump_2_duration_sum,12_hours_ago_temperature,12_hours_ago_precipitation,12_hours_ago_humidity,24_hours_ago_input_flow_rate_first,24_hours_agot_input_flow_rate_last,24_hours_ago_input_flow_rate_mean,24_hours_ago_reservoir_level_percentage_first,24_hours_ago_reservoir_level_percentage_last,24_hours_ago_reservoir_level_percentage_mean,24_hours_ago_output_flow_rate_first,24_hours_ago_output_flow_rate_last,24_hours_ago_output_flow_rate_mean,24_hours_ago_pressure_first,24_hours_ago_pressure_last,24_hours_ago_pressure_mean,24_hours_agot_pump_1_duration_sum,24_hours_agot_pump_2_duration_sum,24_hours_ago_temperature,24_hours_ago_precipitation,24_hours_ago_humidity,26_hours_ago_input_flow_rate_first,26_hours_agot_input_flow_rate_last,26_hours_ago_input_flow_rate_mean,26_hours_ago_reservoir_level_percentage_first,26_hours_ago_reservoir_level_percentage_last,26_hours_ago_reservoir_level_percentage_mean,26_hours_ago_output_flow_rate_first,26_hours_ago_output_flow_rate_last,26_hours_ago_output_flow_rate_mean,26_hours_ago_pressure_first,26_hours_ago_pressure_last,26_hours_ago_pressure_mean,26_hours_agot_pump_1_duration_sum,26_hours_agot_pump_2_duration_sum,26_hours_ago_temperature,26_hours_ago_precipitation,26_hours_ago_humidity,48_hours_ago_input_flow_rate_fir

In [14]:
print(all_features)

['timestamp', 'hour', 'day_of_week', 'week_of_year', 'year', '1_hours_ago_input_flow_rate_first', '1_hours_agot_input_flow_rate_last', '1_hours_ago_input_flow_rate_mean', '1_hours_ago_reservoir_level_percentage_first', '1_hours_ago_reservoir_level_percentage_last', '1_hours_ago_reservoir_level_percentage_mean', '1_hours_ago_output_flow_rate_first', '1_hours_ago_output_flow_rate_last', '1_hours_ago_output_flow_rate_mean', '1_hours_ago_pressure_first', '1_hours_ago_pressure_last', '1_hours_ago_pressure_mean', '1_hours_agot_pump_1_duration_sum', '1_hours_agot_pump_2_duration_sum', '1_hours_ago_temperature', '1_hours_ago_precipitation', '1_hours_ago_humidity', '2_hours_ago_input_flow_rate_first', '2_hours_agot_input_flow_rate_last', '2_hours_ago_input_flow_rate_mean', '2_hours_ago_reservoir_level_percentage_first', '2_hours_ago_reservoir_level_percentage_last', '2_hours_ago_reservoir_level_percentage_mean', '2_hours_ago_output_flow_rate_first', '2_hours_ago_output_flow_rate_last', '2_hours

In [15]:
print(all_non_weather_features)

['timestamp', 'hour', 'day_of_week', 'week_of_year', 'year', '1_hours_ago_input_flow_rate_first', '1_hours_agot_input_flow_rate_last', '1_hours_ago_input_flow_rate_mean', '1_hours_ago_reservoir_level_percentage_first', '1_hours_ago_reservoir_level_percentage_last', '1_hours_ago_reservoir_level_percentage_mean', '1_hours_ago_output_flow_rate_first', '1_hours_ago_output_flow_rate_last', '1_hours_ago_output_flow_rate_mean', '1_hours_ago_pressure_first', '1_hours_ago_pressure_last', '1_hours_ago_pressure_mean', '1_hours_agot_pump_1_duration_sum', '1_hours_agot_pump_2_duration_sum', '2_hours_ago_input_flow_rate_first', '2_hours_agot_input_flow_rate_last', '2_hours_ago_input_flow_rate_mean', '2_hours_ago_reservoir_level_percentage_first', '2_hours_ago_reservoir_level_percentage_last', '2_hours_ago_reservoir_level_percentage_mean', '2_hours_ago_output_flow_rate_first', '2_hours_ago_output_flow_rate_last', '2_hours_ago_output_flow_rate_mean', '2_hours_ago_pressure_first', '2_hours_ago_pressure